In [1]:
!python -V

Python 3.12.1


In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import joblib

In [3]:
df_january = pd.read_parquet("yellow_trips_january.parquet")

In [4]:
df_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## No 1

number of columns

In [5]:
df_january.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [6]:
df_january.shape

(3066766, 19)

## No 2

std of duration

In [7]:
# lookin for std from duration
df_january["duration"] = (
    (df_january["tpep_dropoff_datetime"] - df_january["tpep_pickup_datetime"])
    .dt.total_seconds() / 60
)

duration_std_yellow_trips_january = df_january["duration"].std()
duration_std_yellow_trips_january

np.float64(42.59435124195458)

## No 3

In [8]:
total_before = len(df_january)

# df_filter = df_january.query("duration >= 1 & duration <= 60")
df_filter = df_january[(df_january["duration"] >= 1) & (df_january["duration"] <= 60)]

total_after = len(df_filter)

fraction = total_after / total_before * 100
fraction

98.1220282212598

## No 4

In [ ]:
categorical = ["PULocationID", "DOLocationID"]

df_filter[categorical] = df_filter[categorical].astype(str)

df_filter[categorical].info()

/tmp/ipykernel_68044/2652205662.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[categorical] = df_filter[categorical].astype(str)


<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   PULocationID  object
 1   DOLocationID  object
dtypes: object(2)
memory usage: 68.9+ MB


: 

In [ ]:
train_dicts = df_filter[categorical].to_dict(orient="records")

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

: 

In [ ]:
X_train.shape

(3009173, 515)

In [ ]:
y_train = df_filter['duration'].values
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ], shape=(3009173,))

## No 5

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)

rmse = root_mean_squared_error(y_train, y_train_pred)

print("rmse: ", rmse)

rmse:  7.649261932106969


In [ ]:
joblib.dump(lr, "lr_model.joblib")

## No 6

In [ ]:
df_february = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

df_february["duration"] = (
    (df_february["tpep_dropoff_datetime"] - df_february["tpep_pickup_datetime"])
    .dt.total_seconds() / 60
)

df_filter_feb = df_february[(df_february["duration"] >= 1) & (df_february["duration"] <= 60)]

test_dicts = df_filter_feb[categorical].to_dict(orient="records")
X_test = dv.transform(test_dicts)

y_test = df_filter_feb["duration"].values

y_test_pred = lr.predict(X_test)

test_rmse = root_mean_squared_error(y_test, y_test_pred)

print("test rmse: ", test_rmse)

NameError: name 'pd' is not defined